# 1. Definición del Problema

Comprender qué tipo de vino debo consumir en consideración del meridaje y mi perfil sociodemográfico.

# 2. Obtención del conjunto de datos

In [3]:
!pip install selenium
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Obtención de url de productos de la CAV - URL guardadas en CSV

import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup as soup

# Obtener la data de los vinos desde la página de la CAV/tienda
data = []
driver = webdriver.Chrome()

for x in range(50):
    url = f"https://cav.cl/tienda?q=&hPP=20&idx=products&p={x}&fR%5Bfamily.name%5D%5B0%5D=Vinos&is_v=1"
    driver.get(url)
    
    try:
        age_verification_popup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'modalAge'))
        )
        yes_button = age_verification_popup.find_element(By.XPATH, ".//button[contains(., 'SI')]")
        yes_button.click()
    except (NoSuchElementException, TimeoutException):
        pass

    page_soup = soup(driver.page_source, 'html.parser')
    
    info = []
    for article in page_soup.find_all("article"):
        id = article.attrs.get("data-id")
        sku = article.attrs.get("data-sku")
        name = article.attrs.get("data-name")
        brand = article.attrs.get("data-brand")
        category = article.attrs.get("data-category")
        variant = article.attrs.get("data-variant")
        ref = article.find("a").attrs.get("href")
        info.append([id, sku, name, brand, category, variant, ref])
    
    data.append(info)
    
    # Guardar datos en un archivo CSV después de cada iteración
    df_partial = pd.DataFrame(info, columns=['id', 'sku', 'name', 'brand', 'category', 'variant', 'ref'])
    df_partial.drop_duplicates("id", inplace=True)
    df_partial.to_csv(f"data//raw//vinos_parte_{x}.csv", sep=',', mode='w', index=False)

driver.quit()

# Reshape la data
reshaped_data = [item for sublist in data for item in sublist]

# Crear el DataFrame
df = pd.DataFrame(reshaped_data, columns=['id', 'sku', 'name', 'brand', 'category', 'variant', 'ref'])

# Eliminar los duplicados
df.drop_duplicates("id", inplace=True)

# Guardar CSV de los vinos encontrados
df.to_csv("data//raw//vinos.csv", sep=',', mode='w', index=False)

# Borrar los pack de vinos
df_filtered = df.drop(axis=0, index=df.index[:8])

# Completar la URL en la columna 'ref'
df_filtered['ref'] = 'https://cav.cl/' + df_filtered['ref'].astype(str)

# Guardar CSV solo vinos
df_filtered.to_csv("url_products.csv", sep=',', mode='w', index=False)


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /home/vscode/.cache/selenium/chrome/linux64/119.0.6045.105/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x555ac23695e3 <unknown>
#1 0x555ac202c0b7 <unknown>
#2 0x555ac2062e55 <unknown>
#3 0x555ac205fb81 <unknown>
#4 0x555ac20aa47f <unknown>
#5 0x555ac20a0cc3 <unknown>
#6 0x555ac206c0e4 <unknown>
#7 0x555ac206d0ae <unknown>
#8 0x555ac232fce1 <unknown>
#9 0x555ac2333b7e <unknown>
#10 0x555ac231d4b5 <unknown>
#11 0x555ac23347d6 <unknown>
#12 0x555ac2300dbf <unknown>
#13 0x555ac2357748 <unknown>
#14 0x555ac2357917 <unknown>
#15 0x555ac2368773 <unknown>
#16 0x7faa05e3dea7 start_thread


In [ ]:
#  Obtención de detallada de cada producto desde la URL scrapeada 

import requests
from bs4 import BeautifulSoup
import csv

# Leer las URL desde el archivo CSV
url_list = []

with open('product_url_list.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        url = row['URL']
        url_list.append(url)

# Crear una lista para almacenar los datos de cada producto
all_product_data = []

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Acceder a los atributos 'data-*' de la etiqueta 'section'
    product_section = soup.select_one('section.c-product')

    product_id = product_section['data-id']
    product_sku = product_section['data-sku']
    product_name = product_section['data-name']
    product_brand = product_section['data-brand']
    product_category = product_section['data-category']
    product_variant = product_section['data-variant']
    product_price = product_section['data-price']

    # Acceder a la ficha técnica
    wine_table = soup.find('table', class_='table table-striped')

    # Inicializar variables para los campos de la tabla
    wine_data = {
        "vinyard": "",
        "strain": "",
        "harvest": "",
        "origin": "",
        "line": "",
        "format": "",
        "type": "",
        "alcohol_degrees": "",
        "blend": "",
        "guard": "",
        "pairing": ""
    }

    if wine_table:
        # Obtener todas las filas de la tabla
        rows = wine_table.find_all('tr')

        for row in rows:
            header = row.find('th')
            data = row.find('td')

            if header and data:
                header_text = header.text.strip()
                data_text = data.text.strip()

                # Asignar los datos a las categorías correspondientes
                if header_text in wine_data:
                    wine_data[header_text] = data_text

    # Acceder al puntaje y calificaciones
    score_section = product_section.select_one('.c-product__score')

    if score_section is not None:
        lacav_score_element = score_section.select_one('.o-score--lacav .o-score__points')
        if lacav_score_element is not None:
            lacav_score = lacav_score_element.text
        else:
            lacav_score = "No score available"
    else:
        lacav_score = "No score available"

    # description
    description = soup.select_one('.c-product__body p').text.strip()

    # Información de precios y stock
    price_info = soup.select_one('.c-product__price')
    socio_price = price_info.select('p.o-text--price-store')[0].text
    normal_price = price_info.select('p.o-text--small')[0].text
    stock = price_info.select('p.o-text--small')[1].text

    # Acceder a la image_url
    product_image = soup.select_one('div.c-product__thumb img')['src']

    # Acceder a la sección de percepción
    sheet_body = soup.select_one('div.c-sheet__body')
    dl_elements = sheet_body.find_all('dl', class_='c-sheet__feature--chart') if sheet_body else []

    perception_data = {}

    for dl in dl_elements:
        # Acceder al contenido de c-sheet__signify
        signify = dl.select_one('dd.c-sheet__signify').text.strip()

        # Acceder al contenido de c-chart__track
        chart_track = dl.select_one('div.c-chart__track')
        chart_class_element = chart_track.find('div', class_='c-chart__percent') if chart_track else None
        if chart_class_element and 'class' in chart_class_element.attrs:
            chart_class = chart_class_element['class'][1]

        # Mapear chart_class a su valor correspondiente
        chart_class_map = {
            'c-chart__percent--0': 0,
            'c-chart__percent--20': 0.2,
            'c-chart__percent--40': 0.4,
            'c-chart__percent--60': 0.6,
            'c-chart__percent--80': 0.8,
            'c-chart__percent--100': 1
        }
        chart_value = chart_class_map.get(chart_class, None)

        perception_data[signify] = chart_value

    # Crear un diccionario con la información
    product_data = {
        "url_product": url,
        "product_internal_id": product_id,
        "product_code": product_sku,
        "product_name": product_name,
        "product_brand": product_brand,
        "product_category": product_category,
        "variant": product_variant,
        "price": product_price,
        "cav_points": lacav_score,
        "description": description,
        "member_price": socio_price,
        "normal_price": normal_price,
        "stock": stock,
        "image_url": product_image,
        "vinyard": wine_data["vinyard"],
        "strain": wine_data["strain"],
        "harvest": wine_data["harvest"],
        "origin": wine_data["origin"],
        "line": wine_data["line"],
        "format": wine_data["format"],
        "type": wine_data["type"],
        "alcohol_degrees": wine_data["alcohol_degrees"],
        "blend": wine_data["blend"],
        "guard": wine_data["guard"],
        "pairing": wine_data["pairing"],
        **perception_data  # Agregar datos de percepción
    }

    all_product_data.append(product_data)

# Resto del código sin cambios

fieldnames = [
    "url_product",
    "product_internal_id",
    "product_code",
    "product_name",
    "product_brand",
    "product_category",
    "variant",
    "price",
    "cav_points",
    "description",
    "member_price",
    "normal_price",
    "stock",
    "image_url",
    "vinyard",
    "strain",
    "harvest",
    "origin",
    "line",
    "format",
    "type",
    "alcohol_degrees",
    "blend",
    "guard",
    "pairing",
    "Cuerpo",
    "Frescor",
    "Dulzura",
    "Astringencia"
]


# guardr los datos en un archivo CSV
csv_filename = "productos.csv"

with open(csv_filename, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for product_data in all_product_data:
        writer.writerow(product_data)



# 3. Almacenamiento de la información

In [53]:
import pandas as pd

productos = pd.read_csv('../data/raw/productos.csv') 
meridaje = pd.read_csv('../data/raw/meridaje.csv')
ranking = pd.read_csv('../data/raw/brand_shopper_rank.csv') 

In [54]:
import sqlite3

conn = sqlite3.connect("../data/wine_db.db")
productos.to_sql("productos", conn, if_exists="replace", index=False)
meridaje.to_sql("meridaje", conn, if_exists="replace", index=False)
ranking.to_sql("ranking", conn, if_exists="replace", index=False)

3604

In [55]:
query = """
SELECT *
FROM productos
JOIN meridaje
ON productos.variant = meridaje.variant
LEFT JOIN ranking
ON productos.product_brand = ranking.brand
;
"""

df = pd.read_sql_query(query, conn)

#Eliminar filas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

df = df.rename(columns={'Cuerpo':'body','Astringencia':'astringency','Frescor':'freshness','Dulzura':'sweetness'}) 

#Crear tabla "wine_database"
df.to_sql("wine_database", conn, if_exists="replace", index=False)

conn.close()

# 4. Análisis descriptivo

In [56]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data/wine_db.db')

# Leer tabla "wine_database"
query = "SELECT * FROM wine_database;"
df = pd.read_sql_query(query, conn)

conn.close()

In [57]:
df.columns

Index(['url_product', 'product_internal_id', 'product_code', 'product_name',
       'product_brand', 'product_category', 'variant', 'price', 'cav_points',
       'description', 'member_price', 'normal_price', 'stock', 'image_url',
       'vinyard', 'strain', 'harvest', 'origin', 'line', 'format', 'type',
       'alcohol_degrees', 'blend', 'guard', 'pairing', 'body', 'freshness',
       'sweetness', 'astringency', 'brand', 'gender', 'age', 'ses', 'rank'],
      dtype='object')

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44500 entries, 0 to 44499
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   url_product          44500 non-null  object 
 1   product_internal_id  44500 non-null  int64  
 2   product_code         44500 non-null  int64  
 3   product_name         44500 non-null  object 
 4   product_brand        44500 non-null  object 
 5   product_category     44500 non-null  object 
 6   variant              44500 non-null  object 
 7   price                44500 non-null  int64  
 8   cav_points           44500 non-null  object 
 9   description          25755 non-null  object 
 10  member_price         44500 non-null  object 
 11  normal_price         44500 non-null  object 
 12  stock                44500 non-null  object 
 13  image_url            44500 non-null  object 
 14  vinyard              44500 non-null  object 
 15  strain               44500 non-null 

# 5. EDA (Exploratory data analysis)

# 6. Modelado y optimización

# 7. Despliegue